In [1]:
import ee
import time
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='dse-staff')

# Configuration
modis = ee.ImageCollection('MODIS/061/MOD13A1')
gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
hm = ee.ImageCollection('CSP/HM/GlobalHumanModification').mean()
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
slope = ee.Terrain.slope(elevation)
folder_name = f"boundary-project_results_{datetime.now().strftime('%Y%m%d')}"
print(f"All results will be saved to Google Drive folder: {folder_name}")

staticImage = ee.Image.cat([
    gsw.select('max_extent'),
    hm.rename('gHM'),
    elevation,
    slope
])

years = ee.List.sequence(2001, 2021)
gradBandNames = [str(y) for y in range(2001, 2022)]
selectors = ['WDPA_PID', 'transectID', 'pointID', 'max_extent', 'gHM', 'elevation', 'slope'] + gradBandNames

def make_gradient(y):
    ndvi = modis.filter(ee.Filter.calendarRange(y, y, 'year')).select('NDVI').median()
    grad = ndvi.gradient()
    grad_mag = grad.select('x').hypot(grad.select('y')).unmask(-9999)
    return grad_mag.rename(['grad'])

gradientBands = ee.ImageCollection.fromImages(
    years.map(make_gradient)
).toBands()
gradientBands = gradientBands.rename(gradBandNames)
image = staticImage.addBands(gradientBands)

def process_samples(asset_path, chunk_size=50_000, batch_size=10):
    samples = ee.FeatureCollection(asset_path)
    size = samples.size().getInfo()
    nChunks = int((size + chunk_size - 1) // chunk_size)
    tasks = []

    for i in range(nChunks):
        fcChunk = ee.FeatureCollection(samples.toList(chunk_size, i * chunk_size))
        sampled = image.reduceRegions(
            collection=fcChunk,
            reducer=ee.Reducer.first(),
            scale=500
        )
        task = ee.batch.Export.table.toDrive(
            collection=sampled,
            description=f'ndvi_grad_{asset_path.split("_")[-1]}_chunk_{i}',
            fileFormat='CSV',
            selectors=selectors, 
            folder=folder_name
        )
        tasks.append(task)

    for j in range(0, len(tasks), batch_size):
        batch = tasks[j:j + batch_size]
        for t in batch:
            t.start()
        
        batch_start = j + 1
        batch_end = min(j + batch_size, nChunks)
        print(f"  Processing chunks {batch_start}-{batch_end} of {nChunks}...")
        
        while True:
            statuses = [t.status()['state'] for t in batch]
            if all(s in ['COMPLETED', 'FAILED', 'CANCELLED'] for s in statuses):
                print(f"  Completed chunks {batch_start}-{batch_end} of {nChunks}")
                break
            time.sleep(30)

# Sequentially process each asset from 000 to 009
total_assets = 10
for idx in range(total_assets):
    asset = f'projects/dse-staff/assets/chunk_{idx:03d}'
    print(f"\nProcessing asset {idx + 1} of {total_assets}: {asset}")
    process_samples(asset)
    print(f"Asset {idx + 1} of {total_assets} complete")

print(f"\nAll {total_assets} assets processed!")

All results will be saved to Google Drive folder: boundary-project_results_20251219

Processing asset 1 of 10: projects/dse-staff/assets/chunk_000
  Processing chunks 1-10 of 18...
  Completed chunks 1-10 of 18
  Processing chunks 11-18 of 18...
  Completed chunks 11-18 of 18
Asset 1 of 10 complete

Processing asset 2 of 10: projects/dse-staff/assets/chunk_001
  Processing chunks 1-10 of 16...
  Completed chunks 1-10 of 16
  Processing chunks 11-16 of 16...
  Completed chunks 11-16 of 16
Asset 2 of 10 complete

Processing asset 3 of 10: projects/dse-staff/assets/chunk_002
  Processing chunks 1-10 of 19...
  Completed chunks 1-10 of 19
  Processing chunks 11-19 of 19...
  Completed chunks 11-19 of 19
Asset 3 of 10 complete

Processing asset 4 of 10: projects/dse-staff/assets/chunk_003
  Processing chunks 1-10 of 16...
  Completed chunks 1-10 of 16
  Processing chunks 11-16 of 16...
  Completed chunks 11-16 of 16
Asset 4 of 10 complete

Processing asset 5 of 10: projects/dse-staff/assets